<a href="https://colab.research.google.com/github/bmurphy2019/groupProjectAi/blob/main/heartDiseaseAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Brian Murphy R00189335
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint

# import the csv
df = pd.read_csv('/content/cleveland.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

# data cleaning
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())


# assign target to Y everything else to X
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# 80-20 train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# create the model
model = Sequential()
model.add(Dense(30, input_dim=13, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

# train and checkpoint
checkpointer = ModelCheckpoint('heartDisease.h5', monitor='val_acc', mode='max', verbose=2, save_best_only=True)
history=model.fit(X_train, y_train, batch_size=32, epochs=350, validation_data=(X_test, y_test), callbacks=[checkpointer])



Epoch 1/350
1/8 [==>...........................] - ETA: 5s - loss: 0.6931 - acc: 0.6250
Epoch 00001: val_acc improved from -inf to 0.55738, saving model to heartDisease.h5
8/8 [==============================] - 1s 38ms/step - loss: 0.7170 - acc: 0.5331 - val_loss: 0.6924 - val_acc: 0.5574
Epoch 2/350
1/8 [==>...........................] - ETA: 0s - loss: 0.6853 - acc: 0.5000
Epoch 00002: val_acc did not improve from 0.55738
8/8 [==============================] - 0s 8ms/step - loss: 0.6868 - acc: 0.5248 - val_loss: 0.6929 - val_acc: 0.4918
Epoch 3/350
1/8 [==>...........................] - ETA: 0s - loss: 0.6785 - acc: 0.5625
Epoch 00003: val_acc improved from 0.55738 to 0.60656, saving model to heartDisease.h5
8/8 [==============================] - 0s 7ms/step - loss: 0.6823 - acc: 0.5620 - val_loss: 0.6835 - val_acc: 0.6066
Epoch 4/350
1/8 [==>...........................] - ETA: 0s - loss: 0.6760 - acc: 0.5312
Epoch 00004: val_acc did not improve from 0.60656
8/8 [====================

In [2]:
present_model = keras.models.load_model('heartDisease.h5')
print("Accuracy of our model on test data : " , present_model.evaluate(X_test,y_test)[1]*100 , "%")

2/2 [==============================] - 0s 6ms/step - loss: 0.5276 - acc: 0.7705
Accuracy of our model on test data :  77.04917788505554 %


In [4]:
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(keras.models.load_model('heartDisease.h5'))
tflite_model = converter.convert()

# Save the model.
with open('heartDisease.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpax_zwj3z/assets


INFO:tensorflow:Assets written to: /tmp/tmpax_zwj3z/assets
